In [9]:
#Monter Google Drive
from google.colab import drive
import os
import numpy as np
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler




drive.mount('/content/drive', force_remount=True)


# --- 1. CONFIGURATION ---
base_dir = '/content/drive/MyDrive/Colab Notebooks/Projet-ITI'
results_dir = os.path.join(base_dir, 'results')


input_dir = os.path.join(results_dir, 'task3')


output_dir = os.path.join(results_dir, 'task4')
os.makedirs(output_dir, exist_ok=True) # Création du dossier task4

# --- CHEMINS DES FICHIERS ---

input_histograms_path = os.path.join(input_dir, 'bovw_histograms.npy')

# 2. Les fichiers OUTPUT (Écriture dans task4)
output_pca_features_path = os.path.join(output_dir, 'X_train_pca.npy')
output_pca_model_path = os.path.join(output_dir, 'pca_model.pkl')
output_scaler_path = os.path.join(output_dir, 'scaler_model.pkl')

print("✅ Configuration terminée.")
print(f"📂 Lecture des données depuis le dossier INPUT : {input_dir}")
print(f"📂 Sauvegarde des résultats dans le dossier OUTPUT : {output_dir}")

🗑️ Removing existing /content/drive/MyDrive to ensure clean mount...
Mounted at /content/drive
✅ Configuration terminée.
📂 Lecture des données depuis le dossier INPUT : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/task3
📂 Sauvegarde des résultats dans le dossier OUTPUT : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/task4


In [10]:
# --- 2. CHARGEMENT ---
print("📥 Chargement des histogrammes BoVW...")

if os.path.exists(input_histograms_path):
    X = np.load(input_histograms_path)
    print(f"✅ Données chargées avec succès.")
    print(f"   -> Forme originale (N_images, N_clusters) : {X.shape}")
    # Exemple attendu : (5000, 500) si vous avez 5000 images et K=500
else:
    raise FileNotFoundError(f"❌ Fichier introuvable : {input_histograms_path}")

📥 Chargement des histogrammes BoVW...
✅ Données chargées avec succès.
   -> Forme originale (N_images, N_clusters) : (5000, 500)


In [11]:
# --- 3. NORMALISATION ---
print("⚖️ Standardisation des données (Requis pour la PCA)...")

# On crée le scaler
scaler = StandardScaler()

# On "fit" (calcule moyenne/écart-type) et on "transform" (applique) en même temps
X_scaled = scaler.fit_transform(X)

print(f"✅ Données normalisées.")
print(f"   -> Moyenne globale (devrait être proche de 0) : {np.mean(X_scaled):.2f}")
print(f"   -> Écart-type global (devrait être proche de 1) : {np.std(X_scaled):.2f}")

⚖️ Standardisation des données (Requis pour la PCA)...
✅ Données normalisées.
   -> Moyenne globale (devrait être proche de 0) : 0.00
   -> Écart-type global (devrait être proche de 1) : 1.00


In [12]:
# --- 4. PCA & SAUVEGARDE ---
variance_to_keep = 0.95 # On garde 95% de l'information
print(f"📉 Entraînement de la PCA (Conservation de {variance_to_keep*100}% de variance)...")

# Création et entraînement du modèle PCA
pca = PCA(n_components=variance_to_keep, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Affichage des résultats
n_dims_before = X.shape[1]
n_dims_after = X_pca.shape[1]
print(f"\n✅ PCA terminée !")
print(f"   -> Dimensions avant : {n_dims_before}")
print(f"   -> Dimensions après : {n_dims_after}")
print(f"   -> Gain : {n_dims_before - n_dims_after} dimensions supprimées.")

# --- SAUVEGARDE ---
print("\n💾 Sauvegarde des fichiers...")

# 1. Le Scaler (INDISPENSABLE pour traiter les futures images de test)
with open(output_scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"   -> Scaler sauvegardé : scaler_model.pkl")

# 2. Le Modèle PCA (INDISPENSABLE pour réduire les futures images de test)
with open(output_pca_model_path, 'wb') as f:
    pickle.dump(pca, f)
print(f"   -> Modèle PCA sauvegardé : pca_model.pkl")

# 3. Les Données Finales (Pour l'entraînement à l'étape 5)
np.save(output_pca_features_path, X_pca)
print(f"   -> Features réduites (X) sauvegardées : X_train_pca.npy")

print("\n🎉 Étape 4 Terminée. Tout est prêt pour l'apprentissage !")

📉 Entraînement de la PCA (Conservation de 95.0% de variance)...

✅ PCA terminée !
   -> Dimensions avant : 500
   -> Dimensions après : 433
   -> Gain : 67 dimensions supprimées.

💾 Sauvegarde des fichiers...
   -> Scaler sauvegardé : scaler_model.pkl
   -> Modèle PCA sauvegardé : pca_model.pkl
   -> Features réduites (X) sauvegardées : X_train_pca.npy

🎉 Étape 4 Terminée. Tout est prêt pour l'apprentissage !
